In [50]:
!pip install librosa matplotlib spafe torch

In [51]:
#run
import os
import librosa
import librosa.display
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline

from collections import defaultdict
from spafe.utils import vis
from spafe.features.lfcc import lfcc

In [52]:
import torch
import torch.nn as nn
import math

class SelfAttention(nn.Module):

  def __init__(self, embed_size, num_heads):

    super(SelfAttention, self).__init__()

    self.embed_size = embed_size
    self.num_heads = num_heads
    self.head_dim = embed_size // num_heads

    assert (self.head_dim * num_heads == embed_size), "Embed size needs to be divisible by heads"

    self.values = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias = False)
    self.fc_out = nn.Linear(num_heads * self.head_dim, embed_size)


  def forward(self, values, keys, query, mask):
    N = query.shape[0]
    value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

    # Split embedding into self.num_heads pieces
    values = values.reshape(N, value_len, self.num_heads, self.head_dim)
    keys = keys.reshape(N, key_len, self.num_heads, self.head_dim)
    queries = query.reshape(N, query_len, self.num_heads, self.head_dim)

    values = self.values(values)
    keys = self.keys(keys)
    queries = self.queries(queries)

    energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]) # MatMul Q and K
    # queries shape: (N, query_len, heads, heads_dim)
    # keys shape: (N, query_len, heads, heads_dim)
    # energy shape: (N, heads, query_len, key_len)

    if mask is not None:
      energy = energy.masked_fill(mask == 0, float("-1e20"))

    attention = torch.softmax(energy / (self.embed_size ** 0.5), dim = 3)

    out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
      N, query_len, self.num_heads * self.head_dim
    )
 
    out = self.fc_out(out)

    return out


In [53]:
class PositionalEncoding(nn.Module):

  def __init__(self, embed_size, dropout, max_len = 5000):
    super().__init__()
    print('Embed Size', embed_size)

    self.dropout = nn.Dropout(p = dropout)

    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, embed_size, 2) * (-math.log(10000.0) / embed_size))
    position_encoding = torch.zeros(max_len, 1, embed_size)
    position_encoding[:, 0, 0::2] = torch.sin(position * div_term)
    position_encoding[:, 0, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', position_encoding)

  def forward(self, x):
    x = x + self.position_encoding[:x.size(0)]
    return self.dropout(x)

In [54]:
class TransformerBlock(nn.Module):

  def __init__(self, embed_size, heads, dropout, forward_expansion):
    super(TransformerBlock, self).__init__()

    self.attention = SelfAttention(embed_size, heads)
    self.norm1 = nn.LayerNorm(embed_size)
    self.norm2 = nn.LayerNorm(embed_size)

    self.feed_forward = nn.Sequential(
      nn.Linear(embed_size, forward_expansion * embed_size),
      nn.ReLU(),
      nn.Linear(forward_expansion * embed_size, embed_size)
    )

    self.dropout = nn.Dropout(dropout)


  def forward(self, value, key, query, mask):
    attention = self.attention(value, key, query, mask)

    x = self.dropout(self.norm1((attention + query)))

    forward = self.feed_forward(x)
    out = self.dropout(self.norm2(x + forward))

    return out

In [55]:
class Encoder(nn.Module):

  def __init__(
    self,
    src_vocab_size,
    embed_size,
    num_layers,
    heads,
    device,
    forward_expansion,
    dropout,
    max_length
  ):
    super(Encoder, self).__init__()
    self.embed_size = embed_size
    self.device = device
    self.position_embedding = PositionalEncoding(embed_size, dropout, src_vocab_size)

    self.layers = nn.ModuleList(
      [
        TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)
      ]
    )

    self.dropout = nn.Dropout(dropout)


  def forward(self, x, mask):
    N, seq_length = x.shape

    positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)

    out = self.dropout(self.position_embedding(positions))

    for layer in self.layers:
      out = layer(out, out, out, mask)

    return out


In [56]:
class Transformer(nn.Module):

  def __init__(
    self,
    src_vocab_size,
    src_pad_index,
    embed_size = 256,
    num_layers = 6,
    forward_expansion = 4,
    heads = 8,
    dropout = 0,
    device = "cuda",
    max_length = 500
  ):
    super(Transformer, self).__init__()

    self.encoder = Encoder(
      src_vocab_size, embed_size, num_layers, heads,
      device, forward_expansion, dropout, max_length)

    self.src_pad_index = src_pad_index
    self.device = device


  def make_src_mask(self, src):
    src_mask = (src != self.src_pad_index).unsqueeze(1).unsqueeze(2)

    # (N, 1, 1, src_len)
    return src_mask.to(self.device)

  
  def forward(self, src, target):
    src_mask = self.make_src_mask(src)
    encoder_output = self.encoder(src, src_mask)

    print(encoder_output)

    return encoder_output

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_layers = 4
src_vocab_size = 256 # X_train.shape[1]
src_pad_index = 0
embed_size = 512 # X_train.shape[2]
num_heads = 4
dropout = 0.1
output_size = 1
batch_size = 64

model = Transformer(
  src_vocab_size,
  src_pad_index,
  embed_size = embed_size,
  dropout = dropout,
  heads = num_heads,
  num_layers = num_layers,
  )

Embed Size 512


In [60]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.000003)

loss_fn = nn.MSELoss()

running_loss = 0.
last_loss = 0.

for i, data in enumerate(training_loader):
  inputs, labels = data

  optimizer.zero_grad()

  outputs = model(inputs)

  loss = loss_fn(outputs, labels)
  loss.backward()

  optimizer.step()

  running_loss += loss.item()

  if i % 10 == 0:
    last_loss = running_loss / 1000
    print(' batch {} loss: {}'.format(i + 1, last_loss))
    running_loss = 0.


print('Loss : ', last_loss)

NameError: name 'training_loader' is not defined

In [ ]:
# References:
# 1. https://www.youtube.com/watch?v=zxQyTK8quyY
# 2. https://www.youtube.com/watch?v=bCz4OMemCcA
# 3. https://www.youtube.com/watch?v=gJ9kaJsE78k